In [1]:
import sys
import os
import mlflow
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from pathlib import Path
from tempfile import TemporaryDirectory
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
from typing import Dict, Tuple
from dataclasses import dataclass

sys.path.append('..')
from src.mlflow_utils import configure_mlflow, get_data, find_latest_run_id_by_experiment_and_stage, load_config

In [2]:
CONFIG = load_config()

In [3]:
@dataclass
class TrainingState:
    best_loss: float = float('inf')
    patience_counter: int = 0
    train_losses: list = None
    val_losses: list = None

In [4]:
class Autoencoder(nn.Module):
    """Autoencoder with skip connections (architecture preserved)"""
    def __init__(self, input_dim: int, config: Dict):
        super().__init__()
        params = config["models"]["autoencoder"]["params"]
        
        # Encoder
        self.enc1 = nn.Linear(input_dim, params["hidden_dims"][0])
        self.enc_act1 = nn.ReLU()
        self.enc2 = nn.Linear(params["hidden_dims"][0], params["hidden_dims"][1])
        self.enc_bn2 = nn.BatchNorm1d(params["hidden_dims"][1])
        self.enc_act2 = nn.ReLU()
        self.enc3 = nn.Linear(params["hidden_dims"][1], params["hidden_dims"][2])
        self.enc_bn3 = nn.BatchNorm1d(params["hidden_dims"][2])
        self.enc_act3 = nn.ReLU()
        self.enc4 = nn.Linear(params["hidden_dims"][2], params["encoding_dim"])
        
        # Decoder with skip connections
        self.dropout = nn.Dropout(params["dropout"])
        self.dec1 = nn.Linear(params["encoding_dim"], params["hidden_dims"][2])
        self.dec_act1 = nn.ReLU()
        self.dec2 = nn.Linear(params["hidden_dims"][2], params["hidden_dims"][1])
        self.dec_act2 = nn.ReLU()
        self.dec3 = nn.Linear(params["hidden_dims"][1], params["hidden_dims"][0])
        self.dec_act3 = nn.ReLU()
        self.dec4 = nn.Linear(params["hidden_dims"][0], input_dim)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Encoder
        enc1_out = self.enc_act1(self.enc1(x))
        enc2_out = self.enc_act2(self.enc_bn2(self.enc2(enc1_out)))
        enc3_out = self.enc_act3(self.enc_bn3(self.enc3(enc2_out)))
        encoding = self.enc4(enc3_out)
        drop = self.dropout(encoding)

        # Decoder with skip connections
        dec1_out = self.dec_act1(self.dec1(drop))
        dec2_in = dec1_out + enc3_out  # Skip connection
        dec2_out = self.dec_act2(self.dec2(dec2_in))
        dec3_in = dec2_out + enc2_out
        dec3_out = self.dec_act3(self.dec3(dec3_in))
        dec4_in = dec3_out + enc1_out
        output = self.dec4(dec4_in) 
        
        return output

class Encoder(nn.Module):
    """Encoder component extraction"""
    def __init__(self, autoencoder: Autoencoder):
        super().__init__()
        self.enc1 = autoencoder.enc1
        self.enc_act1 = autoencoder.enc_act1
        self.enc2 = autoencoder.enc2
        self.enc_bn2 = autoencoder.enc_bn2
        self.enc_act2 = autoencoder.enc_act2
        self.enc3 = autoencoder.enc3
        self.enc_bn3 = autoencoder.enc_bn3
        self.enc_act3 = autoencoder.enc_act3
        self.enc4  = autoencoder.enc4

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.enc_act1(self.enc1(x))
        x = self.enc_act2(self.enc_bn2(self.enc2(x)))
        x = self.enc_act3(self.enc_bn3(self.enc3(x)))
        return self.enc4(x)

In [5]:
class AutoencoderTrainer:
    """Handles autoencoder training process"""
    def __init__(self, config: dict, device: torch.device):
        self.config = config
        self.device = device
        self.training_params = config["models"]["autoencoder"]["training_params"]
        self.state = TrainingState(
            train_losses=[],
            val_losses=[]
        )

    def configure_optimizer(self, model: nn.Module) -> torch.optim.Optimizer:
        """Configure optimizer with parameters from config"""
        return torch.optim.Adam(
            model.parameters(),
            lr=self.training_params["lr"],
            weight_decay=self.training_params["weight_decay"]
        )

    def train_epoch(self, model: nn.Module, loader: DataLoader, 
                   optimizer: torch.optim.Optimizer, criterion: nn.Module) -> float:
        """Single training epoch"""
        model.train()
        epoch_loss = 0.0
        
        for batch in tqdm(loader, desc="Training", leave=False):
            data = batch[0].to(self.device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            epoch_loss += loss.item() * data.size(0)
            
        return epoch_loss / len(loader.dataset)

    def validate_epoch(self, model: nn.Module, loader: DataLoader, 
                      criterion: nn.Module) -> float:
        """Single validation epoch"""
        model.eval()
        epoch_loss = 0.0
        
        with torch.no_grad():
            for batch in tqdm(loader, desc="Validating", leave=False):
                data = batch[0].to(self.device)
                output = model(data)
                loss = criterion(output, data)
                epoch_loss += loss.item() * data.size(0)
                
        return epoch_loss / len(loader.dataset)

    def train(self, model: nn.Module, train_loader: DataLoader, 
             val_loader: DataLoader) -> Tuple[list, list]:
        """Full training loop with early stopping"""
        criterion = nn.SmoothL1Loss()
        optimizer = self.configure_optimizer(model)
        patience = self.training_params["patience"]
        
        with tqdm(range(self.training_params["epochs"]), desc="Training") as epoch_bar:
            for epoch in epoch_bar:
                train_loss = self.train_epoch(model, train_loader, optimizer, criterion)
                val_loss = self.validate_epoch(model, val_loader, criterion)
                
                # Update state
                self.state.train_losses.append(train_loss)
                self.state.val_losses.append(val_loss)
                
                # Update progress bar
                epoch_bar.set_postfix({
                    'Train Loss': f'{train_loss:.4f}',
                    'Val Loss': f'{val_loss:.4f}',
                    'LR': f'{optimizer.param_groups[0]["lr"]:.2e}'
                })
                
                # Log metrics
                mlflow.log_metrics({
                    "train_loss": train_loss,
                    "val_loss": val_loss,
                    "learning_rate": optimizer.param_groups[0]['lr']
                }, step=epoch)
                
                # Check early stopping
                if val_loss < self.state.best_loss:
                    self.state.best_loss = val_loss
                    self.state.patience_counter = 0
                    torch.save(model.state_dict(), "best_model.pth")
                else:
                    self.state.patience_counter += 1
                    if self.state.patience_counter >= patience:
                        break

        return self.state.train_losses, self.state.val_losses

In [6]:
class FeatureEngineer:
    """Handles feature engineering pipeline"""
    def __init__(self, config: dict):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.processed_data = None
        self.non_fraud_data = None

    def load_data(self) -> None:
        """Load processed data from MLflow"""
        preprocessing_run_id = find_latest_run_id_by_experiment_and_stage(
            self.config["experiment_names"]["preprocessing"],
            self.config["run_names"]["preprocessing"]
        )
        if not preprocessing_run_id:
            raise ValueError("No successful preprocessing run found")
            
        self.processed_data = get_data(
            preprocessing_run_id, 
            self.config["dataset"], 
            self.config["artifacts"]["data"]["processed"]
        )

    def filter_non_fraud(self) -> None:
        """Filter non-fraud samples for training"""
        self.non_fraud_data = {
            "train": self._filter_split("X_train", "y_train"),
            "val": self._filter_split("X_val", "y_val")
        }

    def _filter_split(self, features_key: str, target_key: str) -> pd.DataFrame:
        """Filter non-fraud samples from a single split"""
        non_fraud_mask = (self.processed_data[target_key] == 0).values.flatten()
        return self.processed_data[features_key][non_fraud_mask]

    def create_loaders(self) -> Tuple[DataLoader, DataLoader]:
        """Create PyTorch data loaders"""
        train_dataset = TensorDataset(
            torch.tensor(self.non_fraud_data["train"].values, dtype=torch.float32)
        )
        val_dataset = TensorDataset(
            torch.tensor(self.non_fraud_data["val"].values, dtype=torch.float32)
        )
        
        batch_size = self.config["models"]["autoencoder"]["training_params"]["batch_size"]
        return (
            DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count()),
            DataLoader(val_dataset, batch_size=batch_size, num_workers=os.cpu_count())
        )

    def generate_features(self, encoder: nn.Module) -> None:
        """Generate and save encoded features"""
        encoder.eval()
        split_names = self.config["dataset"]["split_names"]

        with torch.no_grad(), TemporaryDirectory() as tmp_dir:
            for split in split_names:
                features_key = f"X_{split}"
                data_tensor = torch.tensor(
                    self.processed_data[features_key].values, 
                    dtype=torch.float32
                ).to(self.device)
                
                encoded = pd.DataFrame(
                    encoder(data_tensor).cpu().numpy(),
                    columns=[f"enc_{i}" for i in range(self.config["models"]["autoencoder"]["params"]["encoding_dim"])]
                )

                enriched = pd.concat([
                    self.processed_data[features_key].reset_index(drop=True),
                    encoded.reset_index(drop=True)
                ], axis=1)
                
                output_path = Path(tmp_dir) / f"{features_key}_enriched.parquet"
                enriched.to_parquet(output_path)
                mlflow.log_artifact(
                    output_path, 
                    f"{self.config['artifacts']['data']['engineered']}/{self.config['dataset']['split_dirs'][split_names.index(split)]}"
                )

In [7]:
def log_training_history(train_losses: list, val_losses: list) -> None:
    """Log training curves to MLflow"""
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Autoencoder Training Progress')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    with TemporaryDirectory() as tmp_dir:
        plot_path = Path(tmp_dir) / "training_history.png"
        plt.savefig(plot_path, bbox_inches='tight')
        mlflow.log_artifact(plot_path)
    plt.close()

In [8]:
def feature_engineering_pipeline(config: Dict) -> None:
    """Main feature engineering workflow"""
    # Initialize components
    engineer = FeatureEngineer(config)
    engineer.load_data()
    engineer.filter_non_fraud()
    train_loader, val_loader = engineer.create_loaders()
    
    # Initialize model
    input_dim = engineer.non_fraud_data["train"].shape[1]
    autoencoder = Autoencoder(input_dim, config).to(engineer.device)
    
    # Log model configuration
    mlflow.log_params({
        "input_dim": input_dim,
        **config["models"]["autoencoder"]["params"],
        "optimizer": "Adam",
        "loss_fn": "SmoothL1Loss"
    })
    
    # Train autoencoder
    trainer = AutoencoderTrainer(config, engineer.device)
    train_losses, val_losses = trainer.train(autoencoder, train_loader, val_loader)
    log_training_history(train_losses, val_losses)
    
    # Load best model and create encoder
    autoencoder.load_state_dict(torch.load("best_model.pth"))
    encoder = Encoder(autoencoder)
    
    # Log encoder model
    mlflow.pytorch.log_model(
        pytorch_model=encoder,
        artifact_path="encoder",
        registered_model_name="FraudEncoder",
        extra_files=["best_model.pth"]
    )
    
    # Generate and log features
    engineer.generate_features(encoder)

In [9]:
if __name__ == "__main__":
    experiment_name = CONFIG["experiment_names"]["feature_engineering"]
    run_name = CONFIG["run_names"]["feature_engineering"]
    
    configure_mlflow(experiment_name)
    
    try:
        with mlflow.start_run(run_name=run_name):
            mlflow.set_tags({
                "stage": "feature_engineering",
                "dataset_type": "tabular",
                "task": "reconstruction"
            })
            
            # Log environment details
            mlflow.log_params({
                "pytorch_version": torch.__version__,
                "cuda_available": torch.cuda.is_available(),
                "device_count": torch.cuda.device_count() if torch.cuda.is_available() else 0
            })
            
            feature_engineering_pipeline(CONFIG)
            mlflow.set_tag("status", "completed")
            print(f"Feature engineering completed. Run ID: {mlflow.active_run().info.run_id}")
            
    except Exception as e:
        mlflow.log_param("error", str(e))
        mlflow.set_tag("status", "failed")
        mlflow.end_run()
        raise

Training:   0%|          | 0/200 [00:00<?, ?it/s]

Training:   0%|          | 0/850 [00:00<?, ?it/s]

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Training:   0%|          | 0/850 [00:00<?, ?it/s]

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

...

Training:   0%|          | 0/850 [00:00<?, ?it/s]

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

2025/02/07 20:13:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'FraudEncoder' already exists. Creating a new version of this model...
Created version '29' of model 'FraudEncoder'.
/home/nick/snap/code/181/.local/share/virtualenvs/fraud-detection-mlops-5wXwr5xg/lib/python3.12/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)


Feature engineering completed. Run ID: 04868aac673b472291c7591c15533fb0


/home/nick/snap/code/181/.local/share/virtualenvs/fraud-detection-mlops-5wXwr5xg/lib/python3.12/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
/home/nick/snap/code/181/.local/share/virtualenvs/fraud-detection-mlops-5wXwr5xg/lib/python3.12/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
